# Dataset Description

## Table Description

https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/data

- csv_filesとparquet_filesは同じ内容
  - （データ容量の関係からparquetを残しcsvを削除した）

- Discussion
  - Understanding completion data: https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/discussion/473950
    - depthと対応するファイルの図がわかりやすい

- depth
  - depth=0 - 特定の`case_id`に直接紐づく静的な特徴量
  - depth=1 - 各`case_id`には、`num_group1`によってindexづけされた関連する履歴コードがある
  - depth=1 - 各`case_id`には、`num_group1`と`num_group2`の両方にindexづけされた関連する履歴コードがある

### Configuration

In [1]:
# モジュールの動的import(import先のファイルが更新されたときに追従する)
%load_ext autoreload
%autoreload 2

In [18]:
import os
from glob import glob
import gc

import numpy as np
import polars as pl

from src.utils.pipeline import Pipeline

In [3]:
ROOT = "/kaggle"
TRAIN_DIR = os.path.join(ROOT, "input", "parquet_files", "train")
TEST_DIR = os.path.join(ROOT, "input", "parquet_files", "test")

In [51]:
# 行列を省略しない
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)

# 文字を省略しない
pl.Config.set_fmt_str_lengths(n=1000)

polars.config.Config

### Read data

In [4]:
def read_file(path, depth=None):
  df = pl.read_parquet(path)
  df = df.pipe(Pipeline.set_table_dtypes)

  # if depth in [1, 2]:
  #   df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

  return df

def read_files(regex_path, depth=None):
  chunks = []
  for path in glob(str(regex_path)):
    chunks.append(pl.read_parquet(path).pipe(Pipeline.set_table_dtypes))

  df = pl.concat(chunks, how="vertical_relaxed")

  # if depth in [1, 2]:
  #   df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

  return df

In [37]:
data_store = {
  "base": read_file(os.path.join(TRAIN_DIR, "train_base.parquet")),
  "depth_0": {
      "static_cb_0": read_file(os.path.join(TRAIN_DIR, "train_static_cb_0.parquet")),
      "static_0": read_files(os.path.join(TRAIN_DIR, "train_static_0_*.parquet")),
  },
  "depth_1": {
      "applprev_1": read_files(os.path.join(TRAIN_DIR, "train_applprev_1_*.parquet"), 1),
      "tax_registry_a_1": read_file(os.path.join(TRAIN_DIR, "train_tax_registry_a_1.parquet"), 1),
      "tax_registry_b_1": read_file(os.path.join(TRAIN_DIR, "train_tax_registry_b_1.parquet"), 1),
      "tax_registry_c_1": read_file(os.path.join(TRAIN_DIR, "train_tax_registry_c_1.parquet"), 1),
      "credit_bureau_b_1": read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_1.parquet"), 1),
      "other_1": read_file(os.path.join(TRAIN_DIR, "train_other_1.parquet"), 1),
      "person_1": read_file(os.path.join(TRAIN_DIR, "train_person_1.parquet"), 1),
      "deposit_1": read_file(os.path.join(TRAIN_DIR, "train_deposit_1.parquet"), 1),
      "debitcard_1": read_file(os.path.join(TRAIN_DIR, "train_debitcard_1.parquet"), 1),
  },
  "depth_2": {
      "credit_bureau_b_2": read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_2.parquet"), 2),
  },
}

In [23]:
# 特徴量の説明データ
feature_def_df = pl.read_csv("/kaggle/input/feature_definitions.csv")

### Base tables

- 基本的な情報
- case_idはuniqueなので、他のテーブルと紐づけて使う

- 各カラム
  - `case_id`: uniqueなcredit caseのID
  - `date_decision`: loanを承認すると決定した日付
  - `WEEK_NUM`: 集計に使用される週番号。testデータでは、WEEK_NUMの最後のtrainingの値から続いていく。partition_key
  - `MONTH`: 月を表す。集計を目的とする。
  - `target`: 目的変数。特定のcredit case(loan)で顧客の判定を一定期間後につけた値。
  - `num_group1`: case_idの履歴レコードに使用されるindex for depth=1, 2
  - `num_group2`: case_idの履歴レコードに使用されるindex for depth=2
    - num_group1と2の順序は重要で、特徴量の定義で明確にされる

**test_base.csvにはtrain_base.csvの90%ほどのcase_idが含まれる**


- depth=0では予測変数は直接特徴量として使用できる
- depth>0だと、各case_idに関連する履歴レコードを単一の特徴に集約する必要があるかも
  - num_group1 or num_group2がperson indexを表す場合、zero indexは特別な意味を持つ
  - num_groupN=0の場合、それはローンを申し込んだ人を表す
- データセットの表記
  - P - DPD(Days past due)を変換する
  - M - カテゴリ変数（マスクされている）
  - A - 金額を変換したもの
  - D - 日付を変換したもの
  - T - 指定なし変換
  - L - 指定なし変換

column名の最後の大文字で示される

In [38]:
data_store["base"].head()

case_id,date_decision,MONTH,WEEK_NUM,target
i64,date,i64,i64,i64
0,2019-01-03,201901,0,0
1,2019-01-03,201901,0,0
2,2019-01-04,201901,0,0
3,2019-01-03,201901,0,0
4,2019-01-04,201901,0,1


In [39]:
data_store["depth_0"]["static_0"].head()

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastrepayingdate_696D,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,date,date,date,f64,f64,f64,str,f64,date,f64,bool,bool,date,date,f64,f64,

In [52]:
tmp_df = feature_def_df.filter(pl.col("Variable").is_in(data_store["depth_0"]["static_0"].columns))

tmp_df

Variable,Description
str,str
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""amtinstpaidbefduel24m_4187115A""","""Number of instalments paid before due date in the last 24 months."""
"""annuity_780A""","""Monthly annuity amount."""
"""annuitynextmonth_57A""","""Next month's amount of annuity."""
"""applicationcnt_361L""","""Number of applications associated with the same email address as the client."""
"""applications30d_658L""","""Number of applications made by the client in the last 30 days."""
"""applicationscnt_1086L""","""Number of applications associated with the same phone number."""
"""applicationscnt_464L""","""Number of applications made in the last 30 days by other clients with the same employer as the applicant."""
"""applicationscnt_629L""","""Number of applications with the same employer in the last 7 days."""
